In [1]:
# create consensus from blocks
# create a graph from blocks
# collapse
# resolve loops and branches (look at length-coverage, maybe mapping ?)
# retrieve sequence ?


# NOTES
            # do sth with repeats: integrate into graph then collapse | del 1 of them | ignore and store information | different types = diffrent approach

In [80]:
import pandas as pd
import re
import networkx as nx

cons='/g/scb/bork/groudko/fuso_medium/sibeliaz/mags/sib_loose1000/alignments/all_cons.fa'
permutations='/g/scb/bork/groudko/fuso_medium/sibeliaz/mags/sib_loose1000/genomes_permutations.txt'
gff='/g/scb/bork/groudko/fuso_medium/sibeliaz/mags/sib_loose1000/blocks_coords.gff'
contigs_fasta='/g/scb/bork/groudko/fuso_medium/contigs.fa'
new_contigs_file='/g/scb/bork/groudko/fuso_medium/sibeliaz/mags/sib_loose1000/alignments/mergedLCB.fasta'

In [74]:
class Block:
    def __init__(self, number, sequence):
        self.number=int(number)
        self.sequence=sequence
        self.length=len(self.sequence)
        self.contigs=list()
    def __str__(self):
        return 'Block '+str(self.number)
    def reverse(self):
        complement={'A': 'T', 'T': 'A', 'G': 'C', 'C': 'G'}
        return ''.join(complement.get(base,base) for base in self.sequence[::-1])
        
class myGraph(nx.Graph):
    def write_edge(self, start, end):
        # add an edge to the graph; if the edge already exists, increase weight of the edge; ignore if there is no start/end in graph
        try:
            if (start, end) not in self.edges():
                self.add_edge(start, end, weight=1)
            else:
                self.edges[start,end]['weight']+=1
        except (ValueError, KeyError):
            pass
    
G=myGraph()

In [88]:
# contigs from block paths

# arguments
# - G : graph
# - paths : max coverage paths in a graph
# - contigs : dictionary with contigs sequences
# - new_contigs_file : path to the output

def get_contigs(G,paths,contigs):

    def block_seq(node):
        if re.search('[AB]',node).group()=='A':
            print('+')
            return blocks[re.match('\d+',node).group()].sequence.strip()
        else:
            print('-')
            a=blocks[re.match('\d+',node).group()].reverse().strip()
            print(blocks[re.match('\d+',node).group()].sequence[:10]+'...'+blocks[re.match('\d+',node).group()].sequence[-10:], a[:10]+'...'+a[-10:])
            return blocks[re.match('\d+',node).group()].reverse().strip()

    def choose_contig(node):
        # choose the longest sequence
        if G.nodes[node]['cps'].get('start'):
            def get_value(di, c):
                if di[c][0]:
                    return di[c][0]
                else:
                    return di[c][1]
            # from pairs of contigs (start&end) chooses actual start fragments, compares contig part length (fragment end position) and picks the the one with its maximum
            contig=next(iter(max([min([G.nodes[node]['cps']['start'],G.nodes[node]['cps']['end']], key=lambda x : get_value(x,c)) \
                                  for c in G.nodes[node]['cps'].get('start',{})], key=lambda x: get_value(x,next(iter(x.keys()))))))
            return contig

    new_contigs=dict()
    contig_num=1
    for path in paths:
        print('______________________________')
        print(contig_num)
        print(path)
        sequence=''

        # -----start
        node=path[0]
        print('start',node)
        print(G.nodes[node]) #
        contig=choose_contig(node)
        # retrieve the sequence of contig part from contigs fasta using position
        if contig:
            print(contig)
            pos=G.nodes[node]['cps']['start'][contig]
            if pos[0]!=pos[1]:
                sequence+=contigs[contig][pos[0]:pos[1]:pos[2]]
        sequence+=block_seq(node)

        # -----middle
        for i in range(2,len(path),2):
            print('current node',path[i])

            # append contig part sequence
            positions_dict=G.nodes[path[i-1]]['cps'].get(path[i])
            if positions_dict:
                print('is cp:',path[i-1], path[i], G.nodes[path[i-1]]['cps'])
                contig=max(positions_dict,key=lambda x: abs(positions_dict.get(x)[0]-positions_dict.get(x)[1]))
                print(contig)
                pos=positions_dict[contig]
                if pos[0]!=pos[1]:
                    sequence+=contigs[contig][pos[0]:pos[1]:pos[2]]
            else:
                print('not cp:',path[i-1], path[i], G.nodes[path[i-1]]['cps'])

            # append block sequence
            sequence+=block_seq(path[i])

        # end - only contig part
        node=path[len(path)-1]
        print('end', node)
        contig=choose_contig(node)
        if contig:
            pos=G.nodes[node]['cps']['end'][contig]
            if pos[0]!=pos[1]:
                sequence+=contigs[contig][pos[0]:pos[1]:pos[2]]

        # write contig to file
        '''new_contigs.write('>contig_'+str(contig_num)+'\n')
        new_contigs.write(sequence+'\n')'''
        

        # write to dict and then to file
        new_contigs[contig_num]=sequence.replace('\n','')
        contig_num+=1
    return new_contigs

def dict_to_fasta(di, fasta):
    with open(fasta,'w') as f:
        for key, value in di.items():
            f.write('>contig_'+str(key)+'\n')
            f.write(value+'\n')

G=build_graph_cp(permutations,df_gff)
new=get_contigs(G,paths,contigs)
dict_to_fasta(new,new_contigs_file)

______________________________
1
['483A', '483B', '235A', '235B', '2A', '2B', '163B', '163A']
start 483A
{'cps': {'start': {'SAMEA3541525.psa_megahit.psb_metabat2.00012:k119_47436': (None, 10, 1), 'SAMN03287620.psa_megahit.psb_metabat2.00029:k119_84967': (None, 2839, -1)}, 'end': {'SAMEA3541525.psa_megahit.psb_metabat2.00012:k119_47436': (9, None, -1), 'SAMN03287620.psa_megahit.psb_metabat2.00029:k119_84967': (2840, None, 1)}}}
SAMEA3541525.psa_megahit.psb_metabat2.00012:k119_47436
+
current node 235A
is cp: 483B 235A {'end': {'SAMEA3541525.psa_megahit.psb_metabat2.00012:k119_47436': (1274, None, 1)}, 'start': {'SAMEA3541525.psa_megahit.psb_metabat2.00012:k119_47436': (None, 1273, -1)}, '235A': {'SAMN03287620.psa_megahit.psb_metabat2.00029:k119_84967': (1445, 1444, -1)}}
SAMN03287620.psa_megahit.psb_metabat2.00029:k119_84967
+
current node 2A
is cp: 235B 2A {'2A': {'SAMEA2737906.psa_megahit.psb_metabat2.00057:k119_29658': (1317, 2727, 1), 'SAMN03287587.psa_megahit.psb_metabat2.00005:k1

______________________________
26
['57A', '57B', '166A', '166B', '588B', '588A', '172B', '172A', '269A', '269B', '334A', '334B', '160A', '160B', '578A', '578B']
start 57A
{'cps': {'end': {'SAMEA2737906.psa_megahit.psb_metabat2.00057:k119_186247': (31951, None, 1), 'SAMEA3541525.psa_megahit.psb_metabat2.00012:k119_52762': (5036, None, 1), 'SAMN03287587.psa_megahit.psb_metabat2.00005:k119_57399': (4550, None, 1), 'SAMN03287620.psa_megahit.psb_metabat2.00029:k119_89833': (522, None, -1)}, 'start': {'SAMEA2737906.psa_megahit.psb_metabat2.00057:k119_186247': (None, 31950, -1), 'SAMEA3541525.psa_megahit.psb_metabat2.00012:k119_52762': (None, 5035, -1), 'SAMN03287587.psa_megahit.psb_metabat2.00005:k119_57399': (None, 4549, -1), 'SAMN03287620.psa_megahit.psb_metabat2.00029:k119_89833': (None, 523, 1)}}}
SAMEA2737906.psa_megahit.psb_metabat2.00057:k119_186247
+
current node 166A
is cp: 57B 166A {'166A': {'SAMEA2737906.psa_megahit.psb_metabat2.00057:k119_186247': (30935, 30935, -1), 'SAMN0328758

current node 251A
is cp: 310A 251A {'end': {'SAMEA2737906.psa_megahit.psb_metabat2.00057:k119_13073': (2882, None, 1), 'SAMEA3541525.psa_megahit.psb_metabat2.00012:k119_26598': (2494, None, -1)}, 'start': {'SAMEA2737906.psa_megahit.psb_metabat2.00057:k119_13073': (None, 2881, -1), 'SAMEA3541525.psa_megahit.psb_metabat2.00012:k119_26598': (None, 2495, 1)}, '251A': {'SAMN03287620.psa_megahit.psb_metabat2.00029:k119_127483': (2343, 1843, -1)}}
SAMN03287620.psa_megahit.psb_metabat2.00029:k119_127483
+
current node 264A
is cp: 251B 264A {'264A': {'SAMEA2737906.psa_megahit.psb_metabat2.00057:k119_42252': (2112, 2890, 1), 'SAMN03287587.psa_megahit.psb_metabat2.00005:k119_33443': (1844, 2613, 1)}, 'start': {'SAMN03287620.psa_megahit.psb_metabat2.00029:k119_127483': (None, 0, 1)}, 'end': {'SAMN03287620.psa_megahit.psb_metabat2.00029:k119_127483': (-1, None, -1)}}
SAMEA2737906.psa_megahit.psb_metabat2.00057:k119_42252
+
current node 473A
is cp: 264B 473A {'end': {'SAMEA2737906.psa_megahit.psb_me

is cp: 389B 526B {'start': {'SAMEA2737906.psa_megahit.psb_metabat2.00057:k119_2938': (None, 5, 1)}, 'end': {'SAMEA2737906.psa_megahit.psb_metabat2.00057:k119_2938': (4, None, -1)}, '526B': {'SAMN03287620.psa_megahit.psb_metabat2.00029:k119_46031': (4381, 4874, 1)}}
SAMN03287620.psa_megahit.psb_metabat2.00029:k119_46031
-
ATAAAAATTG...ATATGGACTT AAGTCCATAT...CAATTTTTAT
current node 383A
is cp: 526A 383A {'start': {'SAMN03287587.psa_megahit.psb_metabat2.00005:k119_36821': (None, 674, 1)}, 'end': {'SAMN03287587.psa_megahit.psb_metabat2.00005:k119_36821': (673, None, -1)}, '383A': {'SAMN03287620.psa_megahit.psb_metabat2.00029:k119_46031': (6502, 7102, 1)}}
SAMN03287620.psa_megahit.psb_metabat2.00029:k119_46031
+
end 383B
______________________________
94
['209A', '209B']
start 209A
{'cps': {'start': {'SAMEA2737906.psa_megahit.psb_metabat2.00057:k119_6696': (None, 0, 1), 'SAMEA3541525.psa_megahit.psb_metabat2.00012:k119_49649': (None, 5127, -1), 'SAMN03287620.psa_megahit.psb_metabat2.00029:

TypeError: '<' not supported between instances of 'NoneType' and 'int'

In [86]:
cons='/g/scb/bork/groudko/fuso_medium/sibeliaz/mags/sample/alignments/all_cons.fa'
permutations='/g/scb/bork/groudko/fuso_medium/sibeliaz/mags/sample/genomes_permutations.txt'
gff='/g/scb/bork/groudko/fuso_medium/sibeliaz/mags/sample/blocks_coords.gff'
contigs_fasta='/g/scb/bork/groudko/fuso_medium/samples/medium_contigs.fa'
new_contigs_file='/g/scb/bork/groudko/fuso_medium/sibeliaz/mags/sample/alignments/mergedLCB.fasta'

df_gff=df_from_gff(gff)
blocks=dict_with_blocks(cons)
contigs=fasta_to_dict(contigs_fasta)

G=build_graph_cp(permutations,df_gff)
paths=traverse(G)
new=get_contigs(G,paths,contigs)
dict_to_fasta(new,new_contigs_file)


______________________________
1
['483A', '483B', '235A', '235B', '2A', '2B', '163B', '163A']
start 483A
SAMEA3541525.psa_megahit.psb_metabat2.00012:k119_47436
+
current node 235A
is cp 483B 235A {'end': {'SAMEA3541525.psa_megahit.psb_metabat2.00012:k119_47436': (1274, None, 1)}, 'start': {'SAMEA3541525.psa_megahit.psb_metabat2.00012:k119_47436': (None, 1273, -1)}, '235A': {'SAMN03287620.psa_megahit.psb_metabat2.00029:k119_84967': (1445, 1444, -1)}}
+
current node 2A
is cp 235B 2A {'2A': {'SAMEA2737906.psa_megahit.psb_metabat2.00057:k119_29658': (1317, 2727, 1), 'SAMN03287587.psa_megahit.psb_metabat2.00005:k119_26608': (2032, 2032, 1)}, 'start': {'SAMN03287620.psa_megahit.psb_metabat2.00029:k119_84967': (None, 26, 1)}, 'end': {'SAMN03287620.psa_megahit.psb_metabat2.00029:k119_84967': (25, None, -1)}}
+
current node 163B
is cp 2B 163B {'2A': {'SAMEA2737906.psa_megahit.psb_metabat2.00057:k119_29658': (3913, 1403, 1), 'SAMN03287587.psa_megahit.psb_metabat2.00005:k119_26608': (3188, 3359, 

is cp 575B 63A {'end': {'SAMN03287587.psa_megahit.psb_metabat2.00005:k119_51526': (3060, None, 1)}, 'start': {'SAMN03287587.psa_megahit.psb_metabat2.00005:k119_51526': (None, 3059, -1)}, '63A': {'SAMN03287620.psa_megahit.psb_metabat2.00029:k119_40037': (3998, 3998, 1)}}
+
current node 396A
is cp 63B 396A {'end': {'SAMEA2737906.psa_megahit.psb_metabat2.00057:k119_170405': (3307, None, 1), 'SAMEA3541525.psa_megahit.psb_metabat2.00012:k119_25708': (4199, None, 1), 'SAMN03287587.psa_megahit.psb_metabat2.00005:k119_16585': (2754, None, 1)}, 'start': {'SAMEA2737906.psa_megahit.psb_metabat2.00057:k119_170405': (None, 3306, -1), 'SAMEA3541525.psa_megahit.psb_metabat2.00012:k119_25708': (None, 4198, -1), 'SAMN03287587.psa_megahit.psb_metabat2.00005:k119_16585': (None, 2753, -1)}, '396A': {'SAMN03287620.psa_megahit.psb_metabat2.00029:k119_40037': (6749, 7535, 1)}}
+
end 396B
______________________________
30
['466A', '466B', '330A', '330B', '22A', '22B', '143B', '143A', '271A', '271B', '39B', '3

start 590A
SAMEA3541525.psa_megahit.psb_metabat2.00012:k119_2961
+
current node 101A
is cp 590B 101A {'end': {'SAMEA3541525.psa_megahit.psb_metabat2.00012:k119_2961': (2660, None, 1)}, 'start': {'SAMEA3541525.psa_megahit.psb_metabat2.00012:k119_2961': (None, 2659, -1)}, '101A': {'SAMN03287620.psa_megahit.psb_metabat2.00029:k119_37765': (1655, 1851, 1)}}
+
current node 390A
is cp 101B 390A {'390A': {'SAMEA2737906.psa_megahit.psb_metabat2.00057:k119_118341': (1408, 1408, 1), 'SAMN03287587.psa_megahit.psb_metabat2.00005:k119_90530': (1868, 1868, -1)}, 'end': {'SAMN03287620.psa_megahit.psb_metabat2.00029:k119_37765': (3205, None, 1)}, 'start': {'SAMN03287620.psa_megahit.psb_metabat2.00029:k119_37765': (None, 3204, -1)}}
+
current node 260B
is cp 390B 260B {'260B': {'SAMEA2737906.psa_megahit.psb_metabat2.00057:k119_118341': (2859, 2859, 1)}, 'start': {'SAMN03287587.psa_megahit.psb_metabat2.00005:k119_90530': (None, 418, 1)}, 'end': {'SAMN03287587.psa_megahit.psb_metabat2.00005:k119_90530': 

start 333A
SAMEA3541525.psa_megahit.psb_metabat2.00012:k119_12485
+
end 333B
______________________________
151
['346A', '346B']
start 346A
SAMEA3541525.psa_megahit.psb_metabat2.00012:k119_19875
+
end 346B
______________________________
152
['601A', '601B']
start 601A
SAMEA3541525.psa_megahit.psb_metabat2.00012:k119_58776
+
end 601B
______________________________
153
['544A', '544B', '569B', '569A']
start 544A
SAMEA3541525.psa_megahit.psb_metabat2.00012:k119_73553
+
current node 569B
is cp 544B 569B {'end': {'SAMEA3541525.psa_megahit.psb_metabat2.00012:k119_73553': (1269, None, 1)}, 'start': {'SAMEA3541525.psa_megahit.psb_metabat2.00012:k119_73553': (None, 1268, -1)}, '569B': {'SAMN03287587.psa_megahit.psb_metabat2.00005:k119_40911': (3252, 3252, -1)}}
-
TTGCTCATAA...TCAATTTTAA TTAAAATTGA...TTATGAGCAA
end 569A
______________________________
154
['413A', '413B', '547A', '547B']
start 413A
SAMN03287587.psa_megahit.psb_metabat2.00005:k119_54704
+
current node 547A
is cp 413B 547A {'end': 

TypeError: '<' not supported between instances of 'NoneType' and 'int'

In [104]:

# choose the longest sequence
if G.nodes[node]['cps'].get('start'):
    def get_value(di, c):
        if di[c][0]:
            print('0', di[c][0])
            return di[c][0]
        else:
            print('1',di[c][1])
            return di[c][1]
    # from pairs of contigs (start&end) chooses actual start fragments, compares contig part length (fragment end position) and picks the the one with its maximum
    contig=next(iter(max([min([G.nodes[node]['cps']['start'],G.nodes[node]['cps']['end']],\
                              key=lambda x : get_value(x,c)) \
                          for c in G.nodes[node]['cps'].get('start',{})], key=lambda x: get_value(x,next(iter(x.keys()))))))
print()
node='483A'
print(G.nodes[node]['cps'].get('start',{}))
print('for c in', G.nodes[node]['cps'].get('start',{}).keys())
#[min([G.nodes[node]['cps']['start'],G.nodes[node]['cps']['end']],key=lambda x : get_value(x,c)) for c in G.nodes[node]['cps'].get('start',{}).keys()]
c='SAMEA3541525.psa_megahit.psb_metabat2.00012:k119_47436'
print(min([G.nodes[node]['cps']['start'],G.nodes[node]['cps']['end']],key=lambda x : get_value(x,c)))

1 10
0 9
1 2839
0 2840
0 9
1 10

{'SAMEA3541525.psa_megahit.psb_metabat2.00012:k119_47436': (None, 10, 1), 'SAMN03287620.psa_megahit.psb_metabat2.00029:k119_84967': (None, 2839, -1)}
for c in dict_keys(['SAMEA3541525.psa_megahit.psb_metabat2.00012:k119_47436', 'SAMN03287620.psa_megahit.psb_metabat2.00029:k119_84967'])
1 10
0 9
{'SAMEA3541525.psa_megahit.psb_metabat2.00012:k119_47436': (9, None, -1), 'SAMN03287620.psa_megahit.psb_metabat2.00029:k119_84967': (2840, None, 1)}


In [76]:
# prepare data

# create a dataframe from gff
def df_from_gff(gff):
    df_gff=pd.read_csv(gff, sep='\t', header=2, names=['Contig','Sibelia', 'LCB','Start', 'End', '1', '2', '3', 'Block'])
    df_gff['Contig']=df_gff['Contig'].str.replace(' ', '.')
    return df_gff

df_gff=df_from_gff(gff)

# create dictionary with blocks
def dict_with_blocks(cons):
    blocks=dict()
    number=0
    sequence=''
    with open(cons, 'r') as file_cons:
        for line in file_cons:
            if line.startswith('>'):
                blocks[number]=Block(number, sequence)
                number=re.search('\d+', line).group().strip()
                sequence=''
            else:
                sequence+=line.strip()
        blocks[number]=Block(number, sequence)
        del blocks[0]
    return blocks

blocks=dict_with_blocks(cons)
        
        
# create dictionary with contigs
def fasta_to_dict(fasta):
    # fasta file into dictionary with id as a key and sequence as a value
    di=dict()
    key='empty'
    value=''
    with open(fasta) as file:
        for line in file:
            if line.startswith('>'):
                di[key]=value
                value=''
                key=line[1:].strip()
            else:
                value+=line.strip()
        di[key]=value
        del di['empty']
    return di

contigs=fasta_to_dict(contigs_fasta)

In [4]:
# create dictionary with contigs as a sequence of contig parts and blocks
# contig: [cp1s:cp1e, b1, cp2s:cp2e, b2, cp3s:cp3e] from gff and genome_permutations
# not really needed
contig_length='end'
repeats=dict()
#ADD CONTIG LENGTH
contig_parts_pos=dict()
with open(permutations, 'r') as permut:
    for line in permut:
        if line.startswith('>'):
            name=line[1:].strip()
        else:
            contig_parts_pos[name]=list()
            previous_end=1
            for block in line.split():
                if block!=r'$':   
                    # Series from gff with the position of the block in the contig
                    contig_part=df_gff.loc[(df_gff['Block']==int(block[1:])) & (df_gff['Contig']==name)]
                    if len(contig_part)==1: # check repeats in block
                        this_start=int(contig_part['Start'])
                        contig_parts_pos[name].append(str(previous_end-1)+':'+str(this_start))
                        contig_parts_pos[name].append(block)
                        previous_end=int(contig_part['End'])
                    else:
                        print(contig_part, len(contig_part), block)
                
                        repeats[name]=line
                        del contig_parts_pos[name]
                        break  
            try:
                contig_parts_pos[name].append(str(previous_end-1)+':'+str(contig_length))
            except KeyError:
                pass

                                              Contig  Sibelia  \
2  SAMEA2737906.psa_megahit.psb_metabat2.00057:k1...  Sibelia   
3  SAMEA2737906.psa_megahit.psb_metabat2.00057:k1...  Sibelia   

                  LCB  Start   End  1  2  3  Block  
2  synteny_block_copy   1576  3014  .  +  .      1  
3  synteny_block_copy     29  1473  .  +  .      1   2 +1
                                                Contig  Sibelia  \
113  SAMEA2737906.psa_megahit.psb_metabat2.00057:k1...  Sibelia   
118  SAMEA2737906.psa_megahit.psb_metabat2.00057:k1...  Sibelia   

                    LCB  Start   End  1  2  3  Block  
113  synteny_block_copy   2728  3830  .  +  .     13  
118  synteny_block_copy   1404  2472  .  +  .     13   2 +13
                                                Contig  Sibelia  \
114  SAMN03287587.psa_megahit.psb_metabat2.00005:k1...  Sibelia   
115  SAMN03287587.psa_megahit.psb_metabat2.00005:k1...  Sibelia   

                    LCB  Start   End  1  2  3  Block  
114  synte

In [84]:
# graph with contig part information

def build_graph_cp(permutations, df_gff):
    
    G=myGraph()
    
    # write contigs parts positions as attributes to graph nodes
    def write_cp(start_node, end_node):
        
        if (start_node and end_node) != None:
            
            if not G.nodes[start_node]['cps'].get(end_node):
                G.nodes[start_node]['cps'][end_node]=dict()
            G.nodes[start_node]['cps'][end_node][contig_name]=(int(previous_end), int(this_start-1), 1)
            #print('start to end', start_node, G.nodes[start_node]['cps'])
            
            if not G.nodes[end_node]['cps'].get(start_node):
                G.nodes[end_node]['cps'][start_node]=dict()
            G.nodes[end_node]['cps'][start_node][contig_name]=(int(this_start-2), int(previous_end-1), -1)
            #print('end to start', end_node, G.nodes[end_node]['cps'])
        
        elif start_node==None:
            
            # start & end for the node
            
            #write_cp(end_node,'start') - ideal case (with functions for direct and reverse writing)
            
            if not G.nodes[end_node]['cps'].get('start'):
                G.nodes[end_node]['cps']['start']=dict()
            G.nodes[end_node]['cps']['start'][contig_name]=(None,int(this_start-1),1)
            #print('start',G.nodes[end_node]['cps'])
            
            #write_cp(end_node,'end') - ideal case
            
            if not G.nodes[end_node]['cps'].get('end'):
                G.nodes[end_node]['cps']['end']=dict()
            G.nodes[end_node]['cps']['end'][contig_name]=(int(this_start-2),None,-1)
            #print('end',G.nodes[end_node]['cps'])
        
        elif end_node==None:
            
            if not G.nodes[start_node]['cps'].get('end'):
                G.nodes[start_node]['cps']['end']=dict()
            G.nodes[start_node]['cps']['end'][contig_name]=(int(previous_end),None,1)
            
            if not G.nodes[start_node]['cps'].get('start'):
                G.nodes[start_node]['cps']['start']=dict()
            G.nodes[start_node]['cps']['start'][contig_name]=(None,int(previous_end-1),-1)
        
        
    # create graph
    with open(permutations, 'r') as permut:
        for line in permut:
            if line.startswith('>'):
                contig_name=line[1:].strip()
            else:
                last=None
                previous_end=0
                #print(line)
                for block in line.split():
                    if not re.search('\$', block):
                        contig_part=df_gff.loc[(df_gff['Block']==int(block[1:])) & (df_gff['Contig']==contig_name)]
                        if len(contig_part)!=1: # check repeats in block
                            contig_part=contig_part.iloc[0]
                            # contig_part.loc[0,'End']=contig_part.loc[len(contig_part)-1,'End']
                            indexes=df_gff.loc[(df_gff['Block']==int(block[1:])) & (df_gff['Contig']==contig_name)].index
                            df_gff=df_gff.drop([indexes[0]])
                        this_start=int(contig_part['Start'])
                        number=block[1:].strip()
                        A=number+'A'
                        B=number+'B'
                        if not A in G.nodes():
                            G.add_node(A, cps=dict())
                            G.add_node(B, cps=dict())
                        G.write_edge(A,B)
                        if block[0]=='+': # orientation=='+'
                            G.write_edge(last,A)
                            write_cp(last,A)
                            #print('A',G.nodes[A])
                            #print('B',G.nodes[B])
                            last=B
                        else: # orientation=='-'
                            G.write_edge(last,B)
                            write_cp(last,B)
                            #print('B',G.nodes[B])
                            #print('A',G.nodes[A])
                            
                            last=A
                        previous_end=int(contig_part['End'])
                # do end
                write_cp(last, None)
                #print('after end','A',G.nodes[A],'B',G.nodes[B])
    return G

G=build_graph_cp(permutations, df_gff)

In [57]:
'''eg=nx.Graph()
eg.add_nodes_from(['a','b'],cps=dict())
eg.nodes['a']['cps']['hey']=1
eg.add_edge('a','b')
print(eg.nodes.data())
print(eg.edges())
eg.add_nodes_from(['a','b'],cps=dict())
print(eg.nodes.data())
print(eg.edges())
'a' in eg.nodes()'''
print(df_gff.head())
print(df_gff.iloc[1])
a=df_gff.drop([1])
a.head()

                                              Contig  Sibelia  \
0  SAMN03287620.psa_megahit.psb_metabat2.00029:k1...  Sibelia   
1  SAMN03287587.psa_megahit.psb_metabat2.00005:k1...  Sibelia   
2  SAMEA2737906.psa_megahit.psb_metabat2.00057:k1...  Sibelia   
3  SAMN03287587.psa_megahit.psb_metabat2.00005:k1...  Sibelia   
4  SAMN03287587.psa_megahit.psb_metabat2.00005:k1...  Sibelia   

                  LCB  Start   End  1  2  3  Block  
0  synteny_block_copy    301  1452  .  -  .      1  
1  synteny_block_copy   1283  2434  .  +  .      1  
2  synteny_block_copy   3040  4191  .  +  .      1  
3  synteny_block_copy   2033  3188  .  +  .      2  
4  synteny_block_copy   3360  4545  .  +  .      2  
Contig     SAMN03287587.psa_megahit.psb_metabat2.00005:k1...
Sibelia                                              Sibelia
LCB                                       synteny_block_copy
Start                                                   1283
End                                            

,Contig,Sibelia,LCB,Start,End,1,2,3,Block
0,SAMN03287620.psa_megahit.psb_metabat2.00029:k1...,Sibelia,synteny_block_copy,301,1452,.,-,.,1
2,SAMEA2737906.psa_megahit.psb_metabat2.00057:k1...,Sibelia,synteny_block_copy,3040,4191,.,+,.,1
3,SAMN03287587.psa_megahit.psb_metabat2.00005:k1...,Sibelia,synteny_block_copy,2033,3188,.,+,.,2
4,SAMN03287587.psa_megahit.psb_metabat2.00005:k1...,Sibelia,synteny_block_copy,3360,4545,.,+,.,2
5,SAMN03287620.psa_megahit.psb_metabat2.00029:k1...,Sibelia,synteny_block_copy,2107,3305,.,-,.,2


In [6]:
# find paths functions

def traverse(G):
    paths=[]
    while len(nx.get_node_attributes(G, 'visited'))!=nx.number_of_nodes(G): # there are free nodes
        path=[]
        # find the start (the most covered edge)
        cov_edge=find_covered_edge(G)
        # find path from the start
        path=find_path(G, cov_edge)
        # write path
        paths.append(path)
    return paths

def find_covered_edge(G):
    # finds a non-visited edge with maximum coverage in the graph
    def filter_node(n1):
        return not G.nodes[n1].get('visited')
    unvisited_weight=nx.get_edge_attributes(nx.subgraph_view(G, filter_node=filter_node),'weight')
    return [key for key,value in unvisited_weight.items() if value == max(unvisited_weight.values())][0]

def find_path(G, edge):
    path=list(edge)
    G.nodes[edge[0]]['visited']=True
    G.nodes[edge[1]]['visited']=True
    for i in [1,0]:
        # move to the next block
        if re.match('\d+',edge[0]).group() != re.match('\d+',edge[1]).group():
            node=nextAB(edge[i])
            G.nodes[node]['visited']=True
            path.append(node)
        else:
            node=edge[i]
        nbc=next_block_candidates(G,node)
        # add nodes to path until there will be no suitable node to add
        while nbc:
            node=[n for n in nbc[0] if n != node][0]
            G.nodes[node]['visited']=True
            path.append(node)
            node=nextAB(node)
            G.nodes[node]['visited']=True
            path.append(node)
            nbc=next_block_candidates(G,node)
        # extend in other direction
        path.reverse()
    return path
        
        # ADD AB MOVE
        
        # CONSIDER WHEN TO ADD FIRST-EDGE NODES
        
def next_block_candidates(G, node): 
    # returns the list of max coverage edges or None
    pull=[n for n in nx.neighbors(G,node) if not G.nodes[n].get('visited')]
    if pull:
        max_weight=max([G.get_edge_data(node,end)['weight'] for end in pull])
        return [(node,end) for end in pull if G.get_edge_data(node,end)['weight']==max_weight]
    else:
        return None
    
def nextAB(node):
    block,orient=re.match('(\d+)([AB])',node).group(1,2)
    if orient=='A':
        orient='B'
    else:
        orient='A'
    return block+orient
    
def add_node(node):
    G.nodes[node]['visited']=True
    path.append(node)

#G=build_graph(permutations)
paths=traverse(G)

In [14]:
'''d={'c1':(3,None,1),'c2':(24,23,-1)}
e={'c3':(1,1,1)}
k=None
#max(d,key=lambda x: abs(d.get(x)[0]-d.get(x)[1]))
a=''
a+=None'''
'''#G.nodes.data('cps')
node='901B'
print(G.nodes[node]['cps']['start'])
print(G.nodes[node]['cps']['end'])
#G.nodes[node]['cps']['start']
#a=[G.nodes[node]['cps']['start'][c][1],G.nodes[node]['cps']['end'][c][0]]
def get_value(di, c):
    print(di,c)
    if di[c][0]:
        return di[c][0]
    else:
        return di[c][1]
next(iter(max([min([G.nodes[node]['cps']['start'],G.nodes[node]['cps']['end']], key=lambda x : get_value(x,c)) \
     for c in G.nodes[node]['cps'].get('start',{})], key=lambda x: get_value(x,next(iter(x.keys()))))))'''
print(contig_parts_pos['SAMN03287620.psa_megahit.psb_metabat2.00029:k119_81593'])
print(G.nodes['358A'])
for path in paths:
    for block in path:
        if block=='358A':
            print(path)
            break

['0:1', '+358', '2694:end']
{'cps': {'start': {'SAMN03287620.psa_megahit.psb_metabat2.00029:k119_2318': (None, 0, 1)}, 'end': {'SAMN03287620.psa_megahit.psb_metabat2.00029:k119_2318': (-1, None, -1)}, '564A': {'SAMN03287620.psa_megahit.psb_metabat2.00029:k119_2318': (1104, 1104, 1)}}, 'visited': True}
['460A', '460B', '38A', '38B', '122B', '122A', '135A', '135B', '431A', '431B', '358A', '358B', '564A', '564B', '111A', '111B', '461A', '461B']


In [61]:
#c='SAMN03287628.psa_megahit.psb_metabat2.00010:k119_48115'
#next(iter(min([G.nodes[node]['cps']['start'],G.nodes[node]['cps']['end']], key=lambda x : get_value(x,c))))
#contigs['763577454-palatinetonsils2.psa_megahit.psb_metabat2.00003:k119_1346']
print(paths[24])
block=1056
print(df_gff.loc[df_gff['Block']==block,'Contig'])
print(df_gff.loc[df_gff['Block']==block,'Contig'].iloc[0])

cg=df_gff.loc[df_gff['Block']==block,'Contig'].iloc[0]
if contig_parts_pos.get(cg):
    print(contig_parts_pos[cg])
else:
    print('repeat')
    print(repeats['SAMN03287628.psa_megahit.psb_metabat2.00010:k119_54513'])
print(blocks['1056'].reverse(),end='')
print(blocks['863'].sequence, end='')
print(blocks['1057'].reverse(), end='')

['1056B', '1056A', '863A', '863B', '1057B', '1057A']
6181    SAMN03287587.psa_megahit.psb_metabat2.00005:k1...
6182    763577454-palatinetonsils2.psa_megahit.psb_met...
Name: Contig, dtype: object
SAMN03287587.psa_megahit.psb_metabat2.00005:k119_93294
['0:120', '+1057', '1171:1176', '-863', '2736:2798', '+1056', '4227:end']
AAAAAATAAAATTTAATATTTATGAAATTCTTTGAGAATAATATTATTCTCAAAGAGTATCATGAATAAAATGTTATAAAAACTAAAATAAAAAATATAAGAAGATAAAGTCGAAGATAAAAAAGAAAATTTAAAAAATAAAGTAATAATTATATAAGATTTAGTAAATAATTTAGAACAAGACTTTGACGTATTTTTATATTAAGCAGATATAAACAAAGATAAAATAAAGATTTTAGTAATGAAAAATAGAGAAGACAATTAGATTATCGTCATATTAAATAATTAAGATGATTGGAATATTGAAAAAGAAATTATAGTAAAAAAAATAACAATTTATGAAATAGATAAAGTAGAAATAAATGAAGAAATTTTATACTTCTTGGTATTATAAAAAAATAAAGAAGATTAAAAAGATAATGAAATGGAAATTTCATCCTTGTTAGAGGTCTGGATTAATTTTATTTATTCCCATTACTATTTAACATATCTTTAAGTCTTTTATATAAATTGGTAAGTTTTATTAAAAAATCGTAATATAAAAGTGATAAATATAAAAGAAGTATAAATAAGAATTAAAGAAATTTATTCAATAATTTGTTATTTAAAAAAAATAAATTCAAATATGAATTTTTTAATGTTAAAAAGAGATATAAAACATTAAAGTGAGAGTA

In [3]:
# prepare data

# create a dataframe from gff
def df_from_gff(gff):
    df_gff=pd.read_csv(gff, sep='\t', header=2, names=['Contig','Sibelia', 'LCB','Start', 'End', '1', '2', '3', 'Block'])
    df_gff['Contig']=df_gff['Contig'].str.replace(' ', '.')
    return df_gff

df_gff=df_from_gff(gff)

# create dictionary with blocks
def dict_with_blocks(cons):
    blocks=dict()
    number=0
    sequence=''
    with open(cons, 'r') as file_cons:
        for line in file_cons:
            if line.startswith('>'):
                blocks[number]=Block(number, sequence)
                number=re.search('\d+', line).group().strip()
                sequence=''
            else:
                sequence+=line.strip()
        blocks[number]=Block(number, sequence)
        del blocks[0]
    return blocks

blocks=dict_with_blocks(cons)
        
        
# create dictionary with contigs
def fasta_to_dict(fasta):
    # fasta file into dictionary with id as a key and sequence as a value
    di=dict()
    key='empty'
    value=''
    with open(fasta) as file:
        for line in file:
            if line.startswith('>'):
                di[key]=value
                value=''
                key=line[1:].strip()
            else:
                value+=line.strip()
        di[key]=value
        del di['empty']
    return di

contigs=fasta_to_dict(contigs_fasta)
    
# create dictionary with contigs as a sequence of contig parts and blocks
# contig: [cp1s:cp1e, b1, cp2s:cp2e, b2, cp3s:cp3e] from gff and genome_permutations
# not really needed
contig_length='end'
repeats=dict()
#ADD CONTIG LENGTH
contig_parts_pos=dict()
with open(permutations, 'r') as permut:
    for line in permut:
        if line.startswith('>'):
            name=line[1:].strip()
        else:
            contig_parts_pos[name]=list()
            previous_end=1
            for block in line.split():
                if block!=r'$':   
                    # Series from gff with the position of the block in the contig
                    contig_part=df_gff.loc[(df_gff['Block']==int(block[1:])) & (df_gff['Contig']==name)]
                    if len(contig_part)==1: # check repeats in block
                        this_start=int(contig_part['Start'])
                        contig_parts_pos[name].append(str(previous_end-1)+':'+str(this_start))
                        contig_parts_pos[name].append(block)
                        previous_end=int(contig_part['End'])
                    else:
                        print(contig_part, len(contig_part), block)
                
                        repeats[name]=line
                        del contig_parts_pos[name]
                        break  
            try:
                contig_parts_pos[name].append(str(previous_end-1)+':'+str(contig_length))
            except KeyError:
                pass

                                              Contig  Sibelia  \
2  SAMEA2737906.psa_megahit.psb_metabat2.00057:k1...  Sibelia   
3  SAMEA2737906.psa_megahit.psb_metabat2.00057:k1...  Sibelia   

                  LCB  Start   End  1  2  3  Block  
2  synteny_block_copy   1576  3014  .  +  .      1  
3  synteny_block_copy     29  1473  .  +  .      1   2 +1
                                                Contig  Sibelia  \
113  SAMEA2737906.psa_megahit.psb_metabat2.00057:k1...  Sibelia   
118  SAMEA2737906.psa_megahit.psb_metabat2.00057:k1...  Sibelia   

                    LCB  Start   End  1  2  3  Block  
113  synteny_block_copy   2728  3830  .  +  .     13  
118  synteny_block_copy   1404  2472  .  +  .     13   2 +13
                                                Contig  Sibelia  \
114  SAMN03287587.psa_megahit.psb_metabat2.00005:k1...  Sibelia   
115  SAMN03287587.psa_megahit.psb_metabat2.00005:k1...  Sibelia   

                    LCB  Start   End  1  2  3  Block  
114  synte

In [4]:
# graph with contig part information

def build_graph_cp(permutations, df_gff):
    
    G=myGraph()
    
    # write contigs parts positions as attributes to graph nodes
    def write_cp(start_node, end_node):
        
        if (start_node and end_node) != None:
            
            if not G.nodes[start_node]['cps'].get(end_node):
                G.nodes[start_node]['cps'][end_node]=dict()
            G.nodes[start_node]['cps'][end_node][contig_name]=(int(previous_end), int(this_start-1), 1)
            
            if not G.nodes[end_node]['cps'].get(start_node):
                G.nodes[end_node]['cps'][start_node]=dict()
            G.nodes[end_node]['cps'][start_node][contig_name]=(int(this_start-2), int(previous_end-1), -1)

        elif start_node==None:
            
            # start & end for the node
            
            #write_cp(end_node,'start') - ideal case (with functions for direct and reverse writing)
            
            if not G.nodes[end_node]['cps'].get('start'):
                G.nodes[end_node]['cps']['start']=dict()
            G.nodes[end_node]['cps']['start'][contig_name]=(None,int(this_start-1),1)
            
            
            #write_cp(end_node,'end') - ideal case
            
            if not G.nodes[end_node]['cps'].get('end'):
                G.nodes[end_node]['cps']['end']=dict()
            G.nodes[end_node]['cps']['end'][contig_name]=(int(this_start-2),None,-1)
        
        elif end_node==None:
            
            if not G.nodes[start_node]['cps'].get('end'):
                G.nodes[start_node]['cps']['end']=dict()
            G.nodes[start_node]['cps']['end'][contig_name]=(int(previous_end),None,1)
            
            if not G.nodes[start_node]['cps'].get('start'):
                G.nodes[start_node]['cps']['start']=dict()
            G.nodes[start_node]['cps']['start'][contig_name]=(None,int(previous_end-1),-1)
        
        
    # create graph
    with open(permutations, 'r') as permut:
        for line in permut:
            if line.startswith('>'):
                contig_name=line[1:].strip()
            else:
                last=None
                previous_end=0
                for block in line.split():
                    if not re.search('\$', block):
                        contig_part=df_gff.loc[(df_gff['Block']==int(block[1:])) & (df_gff['Contig']==contig_name)]
                        if len(contig_part)!=1: # check repeats in block
                            contig_part=contig_part.iloc[0]
                            # contig_part.loc[0,'End']=contig_part.loc[len(contig_part)-1,'End']
                            df_gff.drop(df_gff.index[0])
                        this_start=int(contig_part['Start'])
                        #REPEATS (mask)
                        number=block[1:].strip()
                        A=number+'A'
                        B=number+'B'
                        G.add_nodes_from([A,B], cps=dict())
                        G.write_edge(A,B)
                        if block[0]=='+': # orientation=='+'
                            G.write_edge(last,A)
                            write_cp(last,A)
                            last=B
                        else: # orientation=='-'
                            G.write_edge(last,B)
                            write_cp(last,B)
                            last=A
                        previous_end=int(contig_part['End'])
                # do end
                write_cp(last, None)
    return G

G=build_graph_cp(permutations, df_gff)

'''def write_contig_part(node, contig_part):
    if len(contig_part)==1: # check repeats in block
        this_start=int(contig_part['Start'])
        G.nodes[A]['cps'][last]=str(previous_end-1)+':'+str(this_start))

        previous_end=int(contig_part['End'])
    else:
        print(contig_part, len(contig_part), block)

        repeats[contig_name]=line
        del contigs[contig_name]
        break '''
'''if not G.nodes[last]['cps'][A]:
    G.nodes[last]['cps'][A]=dict()
G.nodes[last]['cps'][A][contig_name]=str(this_start-1)+':'+str(previous_end) 

if not G.nodes[A]['cps'][last]:
    G.nodes[A]['cps'][last]=dict()
G.nodes[A]['cps'][last][contig_name]=str(previous_end)+':'+str(this_start-1) '''
                            

"if not G.nodes[last]['cps'][A]:\n    G.nodes[last]['cps'][A]=dict()\nG.nodes[last]['cps'][A][contig_name]=str(this_start-1)+':'+str(previous_end) \n\nif not G.nodes[A]['cps'][last]:\n    G.nodes[A]['cps'][last]=dict()\nG.nodes[A]['cps'][last][contig_name]=str(previous_end)+':'+str(this_start-1) "

In [30]:
#%%time 
#build_graph_cp(permutations, df_gff)
#G.nodes.data('cps')
df_gff.head()
block='-91'
df_gff.loc[(df_gff['Block']==int(block[1:])) & (df_gff['Contig']=='SAMEA2737906.psa_megahit.psb_metabat2.00057:k119_141452')]

,Contig,Sibelia,LCB,Start,End,1,2,3,Block
778,763577454-palatinetonsils2.psa_megahit.psb_met...,Sibelia,synteny_block_copy,341,2382,.,+,.,91
779,SAMN14270932.psa_megahit.psb_metabat2.00019:k1...,Sibelia,synteny_block_copy,116568,118601,.,+,.,91
780,SAMN03287620.psa_megahit.psb_metabat2.00029:k1...,Sibelia,synteny_block_copy,6377,8418,.,+,.,91
781,SAMN03287611.psa_megahit.psb_metabat2.00017:k1...,Sibelia,synteny_block_copy,47256,49297,.,+,.,91
782,SAMEA3541525.psa_megahit.psb_metabat2.00012:k1...,Sibelia,synteny_block_copy,978,2987,.,-,.,91
783,SAMN03287587.psa_megahit.psb_metabat2.00005:k1...,Sibelia,synteny_block_copy,738,2777,.,+,.,91
784,SAMN03287597.psa_megahit.psb_metabat2.00013:k1...,Sibelia,synteny_block_copy,10407,12395,.,-,.,91
785,SAMN03287628.psa_megahit.psb_metabat2.00010:k1...,Sibelia,synteny_block_copy,9272,11265,.,-,.,91


In [8]:
# descriptions of slices to slices : maybe better to use tuples
'''a='12345678'
a[int('2')::-1]
l='1::3'.split(':')
for i in range(len(l)):
    if l[i]=='':
        l[i]=None
    else:
        l[i]=int(l[i])
print(l)
a[l[0]:l[1]:l[2]]'''
a=1
b=None
(a and b) != None

False

In [9]:
# write blocks directly from permutations file into graph

def build_graph(permutations):
    G=myGraph()
    with open(permutations, 'r') as permut:
        for line in permut:
            if not line.startswith('>'):
                last=None
                for block in line.split():
                    if not re.search('\$', block):
                        number=block[1:].strip()
                        A=number+'A'
                        B=number+'B'
                        G.add_nodes_from([A,B])
                        G.write_edge(A,B)
                        if block[0]=='+': # orientation=='+'
                            G.write_edge(last,A)
                            last=B
                        else: # orientation=='-'
                            G.write_edge(last,B)
                            last=A
    return G

G=build_graph(permutations)

# view graph (to graphml, eg for cytoscape)
nx.write_graphml(G, "/g/scb/bork/groudko/fuso_medium/sibeliaz/mags/sample/alignments/medium_contigs.graphml")

ERROR! Session/line number was not unique in database. History logging moved to new session 91


In [10]:
# with repeats
# **** repeats
a['1']='a'

NameError: name 'a' is not defined

In [6]:
# traverse graph - class
'''class myGraph_traverse(nx.Graph):
    
    def write_edge(self, start, end):
        # add an edge to the graph; if the edge already exists, increase weight of the edge 
        # ignore if there is no start/end in graph
        try:
            if (start, end) not in self.edges():
                self.add_edge(start, end, weight=1)
            else:
                self.edges[start,end]['weight']+=1
        except (ValueError, KeyError):
            pass
    def nextAB(self, node):
        node.visited=True
        block=re.match('(\d+)([AB])',node).group()
        block[AB]
        nx.neighbors(self, node)
    def next_block():
        #from edges
    def traverse(self):
        first_edge=[key for key,value in nx.get_edge_attributes(self,'weight').items() if value == max(di.values())][0]
        used=set(first_edge[0],first_edge[1])
        path=list(first_edge)
        add_to_path
        while edge not in used_edges:
            new_edge
            parent_node=edge
        self.data('weight')
        # start from the edge with highest coverage
        # go by picking the most high-covered edges
        # always choose AB/BA
        # store visited edges
'''
        
'''for u, v, weight in G.edges.data("weight"):
    if weight is not None:
        # Do something useful with the edges
        pass
for n, nbrsdict in G.adjacency():
    for nbr, eattr in nbrsdict.items():
        if "weight" in eattr:
            # Do something useful with the edges
            pass'''

IndentationError: expected an indented block (2239767601.py, line 21)

In [5]:
# find paths functions

def traverse(G):
    paths=[]
    while len(nx.get_node_attributes(G, 'visited'))!=nx.number_of_nodes(G): # there are free nodes
        path=[]
        # find the start (the most covered edge)
        cov_edge=find_covered_edge(G)
        # find path from the start
        path=find_path(G, cov_edge)
        # write path
        paths.append(path)
    return paths

def find_covered_edge(G):
    # finds a non-visited edge with maximum coverage in the graph
    def filter_node(n1):
        return not G.nodes[n1].get('visited')
    unvisited_weight=nx.get_edge_attributes(nx.subgraph_view(G, filter_node=filter_node),'weight')
    return [key for key,value in unvisited_weight.items() if value == max(unvisited_weight.values())][0]

def find_path(G, edge):
    path=list(edge)
    G.nodes[edge[0]]['visited']=True
    G.nodes[edge[1]]['visited']=True
    for i in [1,0]:
        # move to the next block
        if re.match('\d+',edge[0]).group() != re.match('\d+',edge[1]).group():
            node=nextAB(edge[i])
            G.nodes[node]['visited']=True
            path.append(node)
        else:
            node=edge[i]
        nbc=next_block_candidates(G,node)
        # add nodes to path until there will be no suitable node to add
        while nbc:
            node=[n for n in nbc[0] if n != node][0]
            G.nodes[node]['visited']=True
            path.append(node)
            node=nextAB(node)
            G.nodes[node]['visited']=True
            path.append(node)
            nbc=next_block_candidates(G,node)
        # extend in other direction
        path.reverse()
    return path
        
        # ADD AB MOVE
        
        # CONSIDER WHEN TO ADD FIRST-EDGE NODES
        
def next_block_candidates(G, node): 
    # returns the list of max coverage edges or None
    pull=[n for n in nx.neighbors(G,node) if not G.nodes[n].get('visited')]
    if pull:
        max_weight=max([G.get_edge_data(node,end)['weight'] for end in pull])
        return [(node,end) for end in pull if G.get_edge_data(node,end)['weight']==max_weight]
    else:
        return None
    
def nextAB(node):
    block,orient=re.match('(\d+)([AB])',node).group(1,2)
    if orient=='A':
        orient='B'
    else:
        orient='A'
    return block+orient
    
def add_node(node):
    G.nodes[node]['visited']=True
    path.append(node)



'''     
    node.visited=True
    block=re.match('(\d+)([AB])',node).group()
    block[AB]
    nx.neighbors(self, node)


first_edge=[key for key,value in nx.get_edge_attributes(G,'weight').items() if value == max(di.values())][0]
used_edges=set(first_edge) # visited edges ? do I need it ?
path=list(first_edge) # visited nodes
if not (re.match('\d+',first_edge[0]) == re.match('\d+',first_edge[1])):
    nextAB(first_edge[1])
while True:
    node=next_block(node)
    path.append(node)
    node=nextAB(node)
    path.append(node)
    parent=node
'''

"     \n    node.visited=True\n    block=re.match('(\\d+)([AB])',node).group()\n    block[AB]\n    nx.neighbors(self, node)\n\n\nfirst_edge=[key for key,value in nx.get_edge_attributes(G,'weight').items() if value == max(di.values())][0]\nused_edges=set(first_edge) # visited edges ? do I need it ?\npath=list(first_edge) # visited nodes\nif not (re.match('\\d+',first_edge[0]) == re.match('\\d+',first_edge[1])):\n    nextAB(first_edge[1])\nwhile True:\n    node=next_block(node)\n    path.append(node)\n    node=nextAB(node)\n    path.append(node)\n    parent=node\n"

In [6]:
#G=build_graph(permutations)
paths=traverse(G)

In [ ]:
# contigs from block paths

# arguments
# - G : graph
# - paths : max coverage paths in a graph
# - contigs : dictionary with contigs sequences
# - new_contigs_file : path to the output

def write_contigs(G,paths,contigs,new_contigs_file):

    def append_block(node, sequence):
        if re.search('[AB]',node).group()=='A':
            sequence+=blocks[re.match('\d+',node).group()].sequence
        else:
            sequence+=blocks[re.match('\d+',node).group()].reverse()
        return sequence

    def choose_contig(node):
        # choose the longest sequence
        if G.nodes[node]['cps'].get('start'):
            def get_value(di, c):
                if di[c][0]:
                    return di[c][0]
                else:
                    return di[c][1]
            # from pairs of contigs (start&end) chooses actual start fragments, compares contig part length (fragment end position) and picks the the one with its maximum
            contig=next(iter(max([min([G.nodes[node]['cps']['start'],G.nodes[node]['cps']['end']], key=lambda x : get_value(x,c)) \
                                  for c in G.nodes[node]['cps'].get('start',{})], key=lambda x: get_value(x,next(iter(x.keys()))))))
            return contig

    with open(new_contigs_file,'w') as new_contigs:
        
        contig_num=1
        for path in paths:
            print('______________________________')
            print(contig_num)
            print(path)
            sequence=''

            # -----start
            node=path[0]
            print('start',node)
            '''contig=choose_contig(node)
            # retrieve the sequence of contig part from contigs fasta using position
            if contig:
                print(contig)
                pos=G.nodes[node]['cps']['start'][contig]
                if pos[0]!=pos[1]:
                    sequence+=contigs[contig][pos[0]:pos[1]:pos[2]]'''
            sequence=append_block(node, sequence)

            # -----middle
            for i in range(2,len(path),2):
                print('current node',path[i])

                # append contig part sequence
                '''positions_dict=G.nodes[path[i-1]]['cps'].get(path[i])
                if positions_dict:
                    print('is cp',path[i-1], path[i], G.nodes[path[i-1]]['cps'])
                    contig=max(positions_dict,key=lambda x: abs(positions_dict.get(x)[0]-positions_dict.get(x)[1]))
                    pos=positions_dict[contig]
                    if pos[0]!=pos[1]:
                        sequence+=contigs[contig][pos[0]:pos[1]:pos[2]]
                else:
                    print('not cp',path[i-1], path[i], G.nodes[path[i-1]]['cps'])'''

                # append block sequence
                sequence=append_block(path[i], sequence)
            
            # end - only contig part
            node=path[len(path)-1]
            print('end')
            '''contig=choose_contig(node)
            if contig:
                pos=G.nodes[node]['cps']['end'][contig]
                if pos[0]!=pos[1]:
                    sequence+=contigs[contig][pos[0]:pos[1]:pos[2]]'''

            # write contig to file
            '''new_contigs.write('>contig_'+str(contig_num)+'\n')
            new_contigs.write(sequence+'\n')
            contig_num+=1'''
            
            # write to dict and then to file
            


write_contigs(G,paths,contigs,new_contigs_file)
    
#AB
#append(blocks[n].sequence)
#BA
#append(blocks[n].reverse())

'''node=path[0]
positions_dict=G.nodes[node]['cps'].get('start')

if positions:
    # choose the longest sequence
    contig=max(positions_dict,key=lambda x: abs(positions_dict.get(x)[1])) # check each pair start and end for length only
    n for n in positions_dict if abs(n[1]-n[0])==max([])
        max(d,key=d.get)
    pos=
    # retrieve the sequence of contig part from contigs fasta using position
    sequence+=contigs[contig][pos[0]:pos[1]:pos[2]]
    path[0]['start']
except KeyError:
    path
sequence+=blocks[re.match('\d+',node).group()].sequence'''

In [7]:
with open('/g/scb/bork/groudko/fuso_medium/sibeliaz/mags/sib_loose1000/alignments/test.fasta','w') as file:
    num=1
    for path in paths:
        seq=''
        for i in range(0,len(path),2):
            node=path[i]
            #if re.search('[AB]',node).group()=='A':
            seq+=blocks[re.match('\d+',node).group()].sequence.strip()
            #else:
            #   seq+=blocks[re.match('\d+',node).group()].reverse().strip()
            #file.write('>c'+re.match('\d+', path[i]).group()+'\n'+blocks[re.match('\d+', path[i]).group()].sequence+'\n')
        file.write('>c_'+str(num)+'\n')
        file.write(seq+'\n')
        num+=1


In [9]:
for path in paths:
    for block in path:
        if block=='454A':
            print(path)
            break

['1055B', '1055A', '959B', '959A', '811B', '811A', '756B', '756A', '280B', '280A', '404A', '404B', '664B', '664A', '286B', '286A', '244A', '244B', '535B', '535A', '881B', '881A', '792A', '792B', '257A', '257B', '478B', '478A', '25B', '25A', '220A', '220B', '227B', '227A', '568B', '568A', '720B', '720A', '658A', '658B', '885A', '885B', '784B', '784A', '772A', '772B', '326B', '326A', '198B', '198A', '575B', '575A', '663A', '663B', '860A', '860B', '119A', '119B', '765B', '765A', '500A', '500B', '704A', '704B', '861B', '861A', '554A', '554B', '525A', '525B', '160B', '160A', '42B', '42A', '197A', '197B', '853A', '853B', '690B', '690A', '819A', '819B', '976A', '976B', '940A', '940B', '287B', '287A', '301A', '301B', '296A', '296B', '618B', '618A', '117A', '117B', '454A', '454B', '39A', '39B', '261B', '261A', '374B', '374A', '757A', '757B', '555B', '555A', '531B', '531A', '362A', '362B', '508B', '508A', '708A', '708B', '402B', '402A', '679B', '679A', '965A', '965B', '573B', '573A', '577A', '57

In [12]:
G.nodes['454A']

{'cps': {'117B': {'SAMN14270932.psa_megahit.psb_metabat2.00019:k141_8392': (93136,
    92788,
    -1)},
  '39A': {'SAMN14270932.psa_megahit.psb_metabat2.00019:k141_8392': (95492,
    95492,
    1)}},
 'visited': True}

In [13]:
contig_parts_pos['SAMN14270932.psa_megahit.psb_metabat2.00019:k141_8392']

['0:430',
 '-420',
 '1561:4796',
 '-811',
 '5851:5877',
 '-756',
 '6980:6982',
 '-280',
 '8239:8241',
 '+404',
 '9599:9601',
 '-664',
 '11685:11687',
 '-286',
 '12764:12992',
 '+244',
 '15271:15273',
 '-535',
 '17025:17027',
 '+738',
 '18255:18755',
 '+792',
 '20417:20419',
 '+257',
 '22213:22215',
 '-478',
 '24406:25102',
 '-25',
 '26219:26723',
 '+220',
 '29077:29079',
 '-227',
 '30468:30470',
 '-568',
 '31852:31854',
 '-720',
 '33139:33613',
 '+658',
 '35432:35434',
 '+885',
 '36498:36500',
 '-784',
 '38289:38429',
 '+772',
 '39807:39809',
 '-326',
 '40963:40965',
 '-198',
 '43925:44395',
 '-575',
 '45732:45734',
 '+663',
 '47244:48297',
 '+119',
 '52708:53694',
 '-765',
 '57204:57206',
 '+500',
 '59331:59458',
 '+704',
 '60737:60739',
 '-861',
 '62468:62785',
 '+554',
 '64489:64491',
 '+525',
 '65679:65703',
 '-160',
 '67639:67641',
 '-42',
 '69139:69646',
 '+197',
 '71494:72523',
 '-690',
 '78365:78740',
 '+819',
 '80524:80526',
 '+976',
 '84122:84124',
 '+940',
 '85804:85806',
 '

In [63]:
#find_covered_edge(G)
print(find_path(G,find_covered_edge(G)))
#next_block_candidates(G,node)
print(len(nx.get_node_attributes(G, 'visited')), nx.number_of_nodes(G))
'''
edge=('13A', '13B')
path=list(edge)
i=1
G.nodes[edge[i]]['visited']=True
# move to the next block
if re.match('\d+',edge[0]).group()!=re.match('\d+',edge[1]).group():
    node=nextAB(edge[i])
    G.nodes[node]['visited']=True
    path.append(node)
    nbc=next_block_candidates(G,node)
else:
    nbc=next_block_candidates(G,edge[i])'''


['831A', '831B']
['887B', '887A', '831A', '831B', '829B', '829A', '889B', '889A']
1866 2152


"\nedge=('13A', '13B')\npath=list(edge)\ni=1\nG.nodes[edge[i]]['visited']=True\n# move to the next block\nif re.match('\\d+',edge[0]).group()!=re.match('\\d+',edge[1]).group():\n    node=nextAB(edge[i])\n    G.nodes[node]['visited']=True\n    path.append(node)\n    nbc=next_block_candidates(G,node)\nelse:\n    nbc=next_block_candidates(G,edge[i])"

In [76]:
blocks

{'1000': <__main__.Block at 0x7f41839d3c40>,
 '1001': <__main__.Block at 0x7f4183972790>,
 '1002': <__main__.Block at 0x7f4183972e50>,
 '1003': <__main__.Block at 0x7f41839727c0>,
 '1004': <__main__.Block at 0x7f41839727f0>,
 '1005': <__main__.Block at 0x7f4183972700>,
 '1006': <__main__.Block at 0x7f4183972e20>,
 '1007': <__main__.Block at 0x7f4183972c40>,
 '1008': <__main__.Block at 0x7f4183972130>,
 '1009': <__main__.Block at 0x7f4183972190>,
 '100': <__main__.Block at 0x7f41839721f0>,
 '1010': <__main__.Block at 0x7f4183972250>,
 '1011': <__main__.Block at 0x7f41839722b0>,
 '1012': <__main__.Block at 0x7f4183972310>,
 '1013': <__main__.Block at 0x7f4183972370>,
 '1014': <__main__.Block at 0x7f41839723d0>,
 '1015': <__main__.Block at 0x7f4183972430>,
 '1016': <__main__.Block at 0x7f4183972490>,
 '1017': <__main__.Block at 0x7f41839724f0>,
 '1018': <__main__.Block at 0x7f4183972550>,
 '1019': <__main__.Block at 0x7f41839725b0>,
 '101': <__main__.Block at 0x7f4183972610>,
 '1020': <__

In [24]:
node='211A'
#block,orient=re.match('(\d+)([AB])',node).group()
'''if orient=='A':
    orient='B'
else:
    orient='A'
str(block+orient)'''
a,b=re.match('(\d+)([AB])',node).group(1,2)

In [102]:
def next_block(G, node): # chooses the right next node
    pull=[G.get_edge_data(node,n) for n in nx.neighbors(G,node) if (n!=parent) and (not G.nodes[n].get('visited'))]
    di=nx.get_edge_attributes(pull, 'weight')
    print(di)
    edge=[key for key,value in di.items() if value == max(di.values())][0]
    return edge[1]
    #print(pull)
    #max(pull)
parent='121A'
next_block(G,'121B')
G.nodes['747B']

myGraph with 2 nodes and 0 edges
{}


IndexError: list index out of range

In [141]:
def next_block_candidates(G, node): # chooses the right next node
    pull=[n for n in nx.neighbors(G,node) if not G.nodes[n].get('visited')]
    if pull:
        max_weight=max([G.get_edge_data(node,end)['weight'] for end in pull])
        return [(node,end) for end in pull if G.get_edge_data(node,end)['weight']==max_weight]
    else:
        return None
G.nodes['491A']['visited']=True
next_block_candidates(G,'491B')

'he'

In [87]:
#[i for i in nx.neighbors(G,'1A')]
tG=myGraph()
tG.add_nodes_from([1,2,3,4])
tG.add_nodes_from([6,7,8,9],visited=True)
tG.add_edges_from([(1,2),(6,7),(8,9),(2,6),(7,3)], weight=1)
tG.add_edge(3,4,weight=2)
def filter_node(n1):
    return not tG.nodes[n1].get('visited')
a=nx.subgraph_view(tG, filter_node=filter_node)
a.edges.data()


EdgeDataView([(1, 2, {'weight': 1}), (3, 4, {'weight': 2})])

In [158]:
print(nx.get_node_attributes(tG, 'visited'))
print(tG.nodes.data('visited'))
i=0
while nx.get_node_attributes(tG, 'visited'):
    print(i)
    i+=1
    if i==10:
        break

4
[(1, None), (2, None), (3, None), (4, None), (6, True), (7, True), (8, True), (9, True)]
0
1
2
3
4
5
6
7
8
9


In [138]:
i=0
t=[1]
while t:
    i+=1
    if i==5:
        t=None
    elif i==10:
        break
    else:
        print(i)
        

1
2
3
4


In [73]:

tG.nodes[1].get('visited')

In [20]:
def cycle_basis(G, root=None):
    """Returns a list of cycles which form a basis for cycles of G"""
    gnodes = set(G.nodes())
    cycles = []
    while gnodes:  # loop over connected components         
        if root is None:             
                root = gnodes.pop()         
            stack = [root]         
            pred = {root: root}         
            used = {root: set()}         
            while stack:  # walk the spanning tree finding cycles             
                z = stack.pop()  # use last-in so cycles easier to find             
                zused = used[z]             
                for nbr in G[z]:                 
                    if nbr not in used:  # new node                     
                        pred[nbr] = z                     
                        stack.append(nbr)                     
                        used[nbr] = {z}                 
                    elif nbr == z:  # self loops                     
                        cycles.append([z])                 
                    elif nbr not in zused:  # found a cycle                     
                        pn = used[nbr]                     
                        cycle = [nbr, z]                     
                        p = pred[z]                     
                        while p not in pn:                         
                            cycle.append(p)                         
                            p = pred[p]                     
                        cycle.append(p)                     
                        cycles.append(cycle)                     
                        used[nbr].add(z)         
        gnodes -= set(pred)         
        root = None     
    return cycles

TypeError: pprint() missing 1 required positional argument: 'object'

In [16]:
# create a visualisation file
nx.write_graphml(G, '/g/scb/bork/groudko/fuso_medium/sibeliaz/mags/sib_loose1000/alignments/blocks.graphml') 

In [6]:
print(G.edges['874B','111B'])
#print(len(G.edges))

{'weight': 3}


In [4]:
# construct a tree

class Block:
    def __init__(self, number, sequence):
        self.number=int(number)
        self.sequence=sequence
        self.length=len(self.sequence)
        self.contigs=list()
    def __str__(self):
        return 'Block '+str(self.number)
    def reverse(self):
        self.sequence=sequence[::-1]
# pull of blocks, create links to them in a tree
# pull of contig parts?????
        
class Tree:
    def __init__(self, value):
        self.value=value
        self.children=[]
        self.length=0
        
    def add_child(self, child, contig_part):
        #if child (block) == None then only save contig_part
        child_tree=Tree(child)
        self.children.append(child_tree)
        # length of path is length of parents' path + contig part + new block
        child_tree.length=self.length+child.length+contig_part.length
        # ??? first cp then block!
        child_tree.contig_part=contig_part
    
        
        
#def next_block(contig):
    # return next block in the contig
    

#longest_block=Block(1, 'ATGC')
#tree=Tree(longest_block)
# find contigs from this block in file (put them in list/attribute) or they are already in block object
# add new child for each contig
# def (block)
# 
#for contig in contigs:
#    tree.add_child(next_block(contig), contig_part)



In [ ]:
# return a sequence

def find_best_path(tree):
    # traverse a tree to find the longest node path length
    # add checking for consistency by a level (breadth first search, go lower and store the longest?)

In [6]:
# block 552 and 1
s='12 34 567'
s.split()


['12', '34', '567']

In [21]:
class Block:
    def __init__(self, number, sequence):
        self.number=int(number)
        self.sequence=sequence
        self.length=len(self.sequence)
        self.contigs=list()
    def __str__(self):
        return 'Block '+str(self.number)
    def reverse(self):
        self.sequence=sequence[::-1]
        
longest_block=Block(1, 'ATGC')

In [22]:
print(longest_block)

Block 1


In [23]:
d=dict()
d[1]='hi'

In [24]:
print(d[1])

hi


In [52]:
import pandas as pd
gff='/g/scb/bork/groudko/fuso_medium/sibeliaz/mags/sib_loose1000/blocks_coords.gff'
df_gff=pd.read_csv(gff, sep='\t', header=2, names=['Contig','Sibelia', 'LCB','Start', 'End', '1', '2', '3', 'Block'])
df_gff['Contig']=df_gff['Contig'].str.replace(' ', '.')
df_gff.head()


,Contig,Sibelia,LCB,Start,End,1,2,3,Block
0,SAMN03287611.psa_megahit.psb_metabat2.00017:k1...,Sibelia,synteny_block_copy,16821,18256,.,-,.,1
1,SAMN03287620.psa_megahit.psb_metabat2.00029:k1...,Sibelia,synteny_block_copy,86,1521,.,-,.,1
2,SAMEA2737906.psa_megahit.psb_metabat2.00057:k1...,Sibelia,synteny_block_copy,1576,3014,.,+,.,1
3,SAMEA2737906.psa_megahit.psb_metabat2.00057:k1...,Sibelia,synteny_block_copy,29,1473,.,+,.,1
4,SAMN14270932.psa_megahit.psb_metabat2.00019:k1...,Sibelia,synteny_block_copy,46445,47880,.,-,.,1


In [44]:
type(df_gff.iloc[0,7])

str

In [8]:
n='+1'
name='SAMN03287611.psa_megahit.psb_metabat2.00017:k119_107186'
int(df_gff.loc[(df_gff['Block']==int(n[1:])) & (df_gff['Contig']==name), 'Start'])

row=df_gff.loc[(df_gff['Block']==int(n[1:])) & (df_gff['Contig']==name), 'Start']
print(row)
#str(row.loc(,'Start'))+':'+str(row.loc(,'End'))

0    16821
Name: Start, dtype: int64


In [88]:
# construct a graph

import networkx as nx

#def create_graph():
#   G=nx.Graph()
    
class myGraph(nx.Graph):
    def write_edge(self, start, end):
        try:
            if (start, end) not in self.edges():
                self.add_edge(start, end, weight=1)
            else:
                self.edges[start,end]['weight']+=1
        except (ValueError, KeyError):
            pass
    
G=myGraph()
#G.add_nodes_from(blocks.keys()) # blocks.keys()
#G.add_edges_from([('1A', '1B'),('2A','2B')])

'''def write_edge(start, end):
    try:
        if edge not in G:
            G.add_edge(start, end, weight=1)
        else:
            G.edges[start,end]['weight']+=1
    except (ValueError, KeyError):
        pass
'''

#or add blocks
'''def add_block(block, G): 
    # block '(\+|\-)\d+' eg block='+12'
    # writes block into graph

    if re.search('^[\+\-]\d+$',block.strip()):
        orientation=block[0]
        number=block[1:].strip()
        A=number+'A'
        B=number+'B'
        G.add_nodes_from([A,B])
        G.add_edge(A,B)
        if orientation=='+':
            G.write_edge(last,A)
            last=B
        else: # orientation='-'
            G.write_edge(last,B)
            last=A
    else:
        raise Exception('Block names are in the wrong format')
'''      
        #when collapsing merge AB or consider it when traversing 


"def add_block(block, G): \n    # block '(\\+|\\-)\\d+' eg block='+12'\n    # writes block into graph\n\n    if re.search('^[\\+\\-]\\d+$',block.strip()):\n        orientation=block[0]\n        number=block[1:].strip()\n        A=number+'A'\n        B=number+'B'\n        G.add_nodes_from([A,B])\n        G.add_edge(A,B)\n        if orientation=='+':\n            G.write_edge(last,A)\n            last=B\n        else: # orientation='-'\n            G.write_edge(last,B)\n            last=A\n    else:\n        raise Exception('Block names are in the wrong format')\n"

In [95]:
# create consensus from blocks
# sort the blocks by length
# start from the longest one
# build a tree - traverse a tree?
# find the longest path
# ??? resolve ???
# discard used blocks
# retrieve sequence (you can do it in parallel/many seq)
# repeat with the next block

In [6]:
df_gff.loc[(df_gff['Block']==int(n[1:])) & (df_gff['Contig']==name)]

,Contig,Sibelia,LCB,Start,End,1,2,3,Block


In [28]:
print(name)

SAMEA2737906.psa_megahit.psb_metabat2.00057:k119_184342


In [11]:
a=df_gff.loc[(df_gff['Block']==int(n[1:])) & (df_gff['Contig']==name)]
a['Start']

2    1576
3      29
Name: Start, dtype: int64

In [22]:
df_gff.iloc[1250]['Contig']==name.strip()

True

In [1]:
contigs['763577454-palatinetonsils2.psa_megahit.psb_metabat2.00003:k119_1346']

NameError: name 'contigs' is not defined

In [28]:
print(repeats)

{'SAMEA2737906.psa_megahit.psb_metabat2.00057:k119_184342': '+1 +1 $\n', 'SAMEA2737906.psa_megahit.psb_metabat2.00057:k119_29658': '-162 +13 +13 $\n', 'SAMN03287587.psa_megahit.psb_metabat2.00005:k119_26608': '-162 +13 +13 $\n', 'SAMN03287597.psa_megahit.psb_metabat2.00013:k119_371': '-94 +100 +839 -18 +868 +552 -1 -1 $\n', 'SAMN03287611.psa_megahit.psb_metabat2.00017:k119_81802': '+934 -77 -26 +273 -10 -891 -693 +363 +199 +172 -170 +546 -777 +642 +683 -191 -398 -451 -723 -737 -632 +324 -171 -74 +298 +680 +334 -593 -444 -423 +494 -236 +132 +450 +104 -71 +113 -538 -469 -487 +175 -51 +345 -856 -529 -368 -284 +114 -626 -215 +598 -767 +12 +396 -435 -171 +592 -892 +741 -728 -778 +969 +745 +221 -640 -788 +676 -346 +749 -202 -495 +533 -208 +89 -73 +40 -56 -271 $\n', 'SAMN03287628.psa_megahit.psb_metabat2.00010:k119_6357': '-1050 +18 -18 +1050 $\n', 'SAMN03287628.psa_megahit.psb_metabat2.00010:k119_54513': '+552 -1 -1 +616 -589 +228 -612 +335 -109 -212 -323 +342 +226 -80 -247 +431 -804 +168 +5

In [23]:
n='-70'
name='763577454-palatinetonsils2.psa_megahit.psb_metabat2.00003:k119_1346'
len(df_gff.loc[(df_gff['Block']==int(n[1:])) & (df_gff['Contig']==name)])

1

In [29]:
a=set()
for i in '+934 -77 -26 +273 -10 -891 -693 +363 +199 +172 -170 +546 -777 +642 +683 -191 -398 -451 -723 -737 -632 +324 -171 -74 +298 +680 +334 -593 -444 -423 +494 -236 +132 +450 +104 -71 +113 -538 -469 -487 +175 -51 +345 -856 -529 -368 -284 +114 -626 -215 +598 -767 +12 +396 -435 -171 +592 -892 +741 -728 -778 +969 +745 +221 -640 -788 +676 -346 +749 -202 -495 +533 -208 +89 -73 +40 -56 -271'.split():
    if i in a:
        print(i)
    a.add(i)

-171


In [51]:
s='134 '
if re.search('^[\+\-]\d+$',s.strip()):
    print(True)
else:
    raise Exception('Block names are in the wrong format')
    

Exception: Block names are in the wrong format

In [15]:
G.add_edge(None,'1A')

ValueError: None cannot be a node

In [33]:
print(('1A', '1B') in G.edges())

True


In [106]:
print(G.edges.data('weight'))

[('151A', '151B', 1), ('151A', '70B', 32), ('151B', '486B', 20), ('151B', '845A', 4), ('151B', '1063B', 4), ('70A', '70B', 1), ('70A', '250B', 28), ('23A', '23B', 1), ('23A', '206B', 24), ('23A', '229A', 4), ('23B', '37A', 28), ('37A', '37B', 1), ('37B', '365B', 28), ('365A', '365B', 1), ('365A', '127A', 28), ('127A', '127B', 1), ('127B', '182A', 20), ('182A', '182B', 1), ('182B', '219B', 24), ('219A', '219B', 1), ('219A', '209B', 24), ('209A', '209B', 1), ('209A', '580B', 24), ('580A', '580B', 1), ('580A', '652A', 20), ('652A', '652B', 1), ('652A', '227A', 4), ('652B', '888A', 16), ('370A', '370B', 1), ('370A', '617B', 16), ('370A', '475A', 4), ('370A', '30B', 4), ('370B', '834A', 16), ('617A', '617B', 1), ('617A', '30B', 20), ('617B', '475B', 8), ('30A', '30B', 1), ('30A', '123B', 28), ('123A', '123B', 1), ('123A', '44A', 20), ('123A', '93B', 8), ('44A', '44B', 1), ('44B', '93B', 16), ('93A', '93B', 1), ('93A', '433B', 20), ('93A', '499B', 4), ('267A', '267B', 1), ('267A', '620A', 16

In [96]:
print(G)

myGraph with 2152 nodes and 2367 edges


In [90]:
len(contigs)

1396

In [9]:
list(df_gff.loc[df_gff['Block']==695,'Contig'])

['SAMN03287628.psa_megahit.psb_metabat2.00010:k119_16645',
 'SAMN03287597.psa_megahit.psb_metabat2.00013:k119_46447']

In [7]:
print(contigs['SAMN03287628.psa_megahit.psb_metabat2.00010:k119_16645'])

['0:178', '-111', '2895:3210', '-234', '5563:5612', '+27', '7152:7701', '-6', '10060:10540', '-446', '11725:11963', '+453', '13040:13253', '-410', '15112:15223', '-695', '16971:17875', '+415', '19645:20348', '-359', '21418:21538', '-810', '22757:22808', '+179', '23901:23925', '-510', '28649:38055', '-146', '39581:40164', '-458', '41741:41748', '+31', '44550:44552', '+159', '46093:46168', '-17', '47290:47293', '-230', '48621:48657', '+496', '49662:49674', '+65', '50759:54675', '+685', '56079:56345', '-547', '57424:57772', '-514', '60637:61522', '+623', '62794:63464', '+902', '65156:65208', '+806', '68657:0']


In [15]:
blocks['695'].length

1783

In [27]:
s="""'    while gnodes:  # loop over connected components\n'
 '        if root is None:\n'
 '            root = gnodes.pop()\n'
 '        stack = [root]\n'
 '        pred = {root: root}\n'
 '        used = {root: set()}\n'
 '        while stack:  # walk the spanning tree finding cycles\n'
 '            z = stack.pop()  # use last-in so cycles easier to find\n'
 '            zused = used[z]\n'
 '            for nbr in G[z]:\n'
 '                if nbr not in used:  # new node\n'
 '                    pred[nbr] = z\n'
 '                    stack.append(nbr)\n'
 '                    used[nbr] = {z}\n'
 '                elif nbr == z:  # self loops\n'
 '                    cycles.append([z])\n'
 '                elif nbr not in zused:  # found a cycle\n'
 '                    pn = used[nbr]\n'
 '                    cycle = [nbr, z]\n'
 '                    p = pred[z]\n'
 '                    while p not in pn:\n'
 '                        cycle.append(p)\n'
 '                        p = pred[p]\n'
 '                    cycle.append(p)\n'
 '                    cycles.append(cycle)\n'
 '                    used[nbr].add(z)\n'
 '        gnodes -= set(pred)\n'
 '        root = None\n'
 '    return cycles\n'"""
s=s.replace("'","")
s.replace("\n","")


'    while gnodes:  # loop over connected components         if root is None:             root = gnodes.pop()         stack = [root]         pred = {root: root}         used = {root: set()}         while stack:  # walk the spanning tree finding cycles             z = stack.pop()  # use last-in so cycles easier to find             zused = used[z]             for nbr in G[z]:                 if nbr not in used:  # new node                     pred[nbr] = z                     stack.append(nbr)                     used[nbr] = {z}                 elif nbr == z:  # self loops                     cycles.append([z])                 elif nbr not in zused:  # found a cycle                     pn = used[nbr]                     cycle = [nbr, z]                     p = pred[z]                     while p not in pn:                         cycle.append(p)                         p = pred[p]                     cycle.append(p)                     cycles.append(cycle)                     used[nbr].a

In [6]:
G.edges(edge for edge in G.edges.data('weight') if edge['weight'] == max([edge[2] for edge in G.edges.data('weight')])) 

NetworkXError: nbunch is not a node or a sequence of nodes.

In [8]:
[edge['weight'] for edge in G.edges]

TypeError: tuple indices must be integers or slices, not str

In [11]:
[key for key,value in nx.get_edge_attributes(G,'weight').items() if value == max(di.values())]

[('1A', '1B')]

In [13]:
G.edges['1A','1B']

{'weight': 14}

In [23]:
[nbrsdict for n,nbrsdict in G.adjacency()]

[{'151B': {'weight': 8}, '70B': {'weight': 8}},
 {'151A': {'weight': 8},
  '486B': {'weight': 5},
  '845A': {'weight': 1},
  '1063B': {'weight': 1}},
 {'70B': {'weight': 9}, '250B': {'weight': 7}},
 {'70A': {'weight': 9}, '151A': {'weight': 8}},
 {'23B': {'weight': 9}, '206B': {'weight': 6}, '229A': {'weight': 1}},
 {'23A': {'weight': 9}, '37A': {'weight': 7}},
 {'37B': {'weight': 9}, '23B': {'weight': 7}},
 {'37A': {'weight': 9}, '365B': {'weight': 7}},
 {'365B': {'weight': 7}, '127A': {'weight': 7}},
 {'365A': {'weight': 7}, '37B': {'weight': 7}},
 {'127B': {'weight': 8}, '365A': {'weight': 7}},
 {'127A': {'weight': 8}, '182A': {'weight': 5}},
 {'182B': {'weight': 7}, '127B': {'weight': 5}},
 {'182A': {'weight': 7}, '219B': {'weight': 6}},
 {'219B': {'weight': 8}, '209B': {'weight': 6}},
 {'219A': {'weight': 8}, '182B': {'weight': 6}},
 {'209B': {'weight': 8}, '580B': {'weight': 6}},
 {'209A': {'weight': 8}, '219A': {'weight': 6}},
 {'580B': {'weight': 6}, '652A': {'weight': 5}},
 {'

In [35]:
for n,nbrsdict in G['1A'].adjacency['1A']

AttributeError: 'AtlasView' object has no attribute 'adjacency'

In [ ]:
def find_path(G, start):
    path=list(start)
    #if number_block of start != that of end:
    #    move to the AB block
    nbc=[start]
    #add nodes to path until there will be no suitable node to add
    while nbc:
        node=nbc[0]
        G.nodes[node]['visited']=True
        path.append(node)
        node=nextAB(node)
        G.nodes[node]['visited']=True
        path.append(node)
        nbc=next_block_candidates(G,node)
    return path

def traverse(G):
    paths=[]
    while len(get_node_attributes(G, 'visited'))!=number_of_nodes(G): # there are free nodes
        paths=[]
        # find the start (the most covered edge)
        cov_edge=find_covered_edge(G)
        # find path from the start
        path=find_path(G, cov_edge[0])
        # extend in other direction
        path=path.reverse()
        path.append(find_path(G, cov_edge[1]))
        # write path
        paths.append(path)

In [ ]:
# scratch

def get_contigs(G,paths,contigs):

    def block_seq(node):
        if re.search('[AB]',node).group()=='A':
            print('+')
            return blocks[re.match('\d+',node).group()].sequence.strip()
        else:
            print('-')
            a=blocks[re.match('\d+',node).group()].reverse().strip()
            print(blocks[re.match('\d+',node).group()].sequence[:10]+'...'+blocks[re.match('\d+',node).group()].sequence[-10:], a[:10]+'...'+a[-10:])
            return blocks[re.match('\d+',node).group()].reverse().strip()

    def choose_contig(node):
        # choose the longest sequence
        if G.nodes[node]['cps'].get('start'):
            def get_value(di, c):
                if di[c][0]:
                    return di[c][0]
                else:
                    return di[c][1]
            # from pairs of contigs (start&end) chooses actual start fragments, compares contig part length (fragment end position) and picks the the one with its maximum
            contig=next(iter(max([min([G.nodes[node]['cps']['start'],G.nodes[node]['cps']['end']], key=lambda x : get_value(x,c)) \
                                  for c in G.nodes[node]['cps'].get('start',{})], key=lambda x: get_value(x,next(iter(x.keys()))))))
            return contig

    new_contigs=dict()
    contig_num=1
    for path in paths:
        print('______________________________')
        print(contig_num)
        print(path)
        sequence=''

        # -----start
        node=path[0]
        print('start',node)
        contig=choose_contig(node)
        # retrieve the sequence of contig part from contigs fasta using position
        if contig:
            print(contig)
            pos=G.nodes[node]['cps']['start'][contig]
            if pos[0]!=pos[1]:
                sequence+=contigs[contig][pos[0]:pos[1]:pos[2]]
        sequence+=block_seq(node)

        # -----middle
        for i in range(2,len(path),2):
            print('current node',path[i])

            # append contig part sequence
            positions_dict=G.nodes[path[i-1]]['cps'].get(path[i])
            if positions_dict:
                print('is cp',path[i-1], path[i], G.nodes[path[i-1]]['cps'])
                contig=max(positions_dict,key=lambda x: abs(positions_dict.get(x)[0]-positions_dict.get(x)[1]))
                pos=positions_dict[contig]
                if pos[0]!=pos[1]:
                    sequence+=contigs[contig][pos[0]:pos[1]:pos[2]]
            else:
                print('not cp',path[i-1], path[i], G.nodes[path[i-1]]['cps'])

            # append block sequence
            sequence+=block_seq(path[i])

        # end - only contig part
        node=path[len(path)-1]
        print('end', node)
        contig=choose_contig(node)
        if contig:
            pos=G.nodes[node]['cps']['end'][contig]
            if pos[0]!=pos[1]:
                sequence+=contigs[contig][pos[0]:pos[1]:pos[2]]

        # write contig to file
        '''new_contigs.write('>contig_'+str(contig_num)+'\n')
        new_contigs.write(sequence+'\n')'''
        

        # write to dict and then to file
        new_contigs[contig_num]=sequence.replace('\n','')
        contig_num+=1
    return new_contigs

def dict_to_fasta(di, fasta):
    with open(fasta,'w') as f:
        for key, value in di.items():
            f.write('>contig_'+str(key)+'\n')
            f.write(value+'\n')

G=build_graph_cp(permutations,df_gff)
new=get_contigs(G,paths,contigs)
dict_to_fasta(new,new_contigs_file)

In [28]:
with open('/g/scb/bork/groudko/fuso_medium/sibeliaz/mags/sample/alignments/cps.fa','w') as cps_file:
    for c,pos in contig_parts_pos.items():
        seq=contigs[c.strip()]
        for bl in pos:
            if ':' in bl:
                a,b=bl.split(':')
                if b == 'end':
                    a=int(a)
                    b=None
                else:
                    a,b=int(a),int(b)
                    if len(seq[a:b])>=100:
                        cps_file.write('>cps_'+c+'_'+bl+'\n')
                        cps_file.write(seq[a:b]+'\n')

In [31]:
with open('/g/scb/bork/groudko/fuso_medium/sibeliaz/mags/sample/alignments/contigs_wo_blocks.fa','w') as f:
    for c in contigs:
        if c not in contig_parts_pos.keys():
            f.write('>'+c+'\n')
            f.write(contigs[c]+'\n')


In [ ]:
a='start'
b=''
if b == 'end':
    b=None
print(a,b)

In [1]:
final={'SAMEA3541525.psa_megahit.psb_metabat2.00012:k119_54396_4': {'TIGR02493': [[4, 234]]}, 'SAMEA2737906.psa_megahit.psb_metabat2.00057:k119_140652_3': {'TIGR01143': [[204, 606]], 'TIGR01656': [[3, 150]], 'TIGR00213': [[3, 154]]}, 'SAMN03287587.psa_megahit.psb_metabat2.00005:k119_74011_5': {'TIGR00457': [[2, 461]]}, 'SAMN03287620.psa_megahit.psb_metabat2.00029:k119_125320_4': {'TIGR00457': [[2, 461]]}, 'SAMEA3541525.psa_megahit.psb_metabat2.00012:k119_5248_1': {'TIGR00457': [[2, 350]]}, 'SAMEA3541525.psa_megahit.psb_metabat2.00012:k119_2244_2': {'TIGR01292': [[5, 307]], 'PF13192.1': [[468, 543]]}, 'SAMN03287620.psa_megahit.psb_metabat2.00029:k119_86524_2': {'TIGR01292': [[5, 307]], 'PF13192.1': [[468, 543]]}, 'SAMEA3541525.psa_megahit.psb_metabat2.00012:k119_45615_5': {'TIGR01510': [[4, 156]]}, 'SAMEA2737906.psa_megahit.psb_metabat2.00057:k119_71631_12': {'TIGR01510': [[4, 156]]}, 'SAMEA2737906.psa_megahit.psb_metabat2.00057:k119_168232_5': {'TIGR02618': [[8, 456]]}, 'SAMEA3541525.psa_megahit.psb_metabat2.00012:k119_1359_4': {'TIGR02618': [[8, 456]]}, 'SAMEA3541525.psa_megahit.psb_metabat2.00012:k119_56142_8': {'TIGR03596': [[6, 287]]}, 'SAMEA2737906.psa_megahit.psb_metabat2.00057:k119_172198_2': {'TIGR00068': [[2, 122]]}, 'SAMN03287620.psa_megahit.psb_metabat2.00029:k119_72255_3': {'TIGR00068': [[2, 122]]}, 'SAMEA3541525.psa_megahit.psb_metabat2.00012:k119_20818_13': {'TIGR00068': [[2, 122]]}, 'SAMN03287620.psa_megahit.psb_metabat2.00029:k119_81593_5': {'TIGR01085': [[19, 298]]}, 'SAMEA3541525.psa_megahit.psb_metabat2.00012:k119_81368_7': {'TIGR01085': [[19, 299]]}, 'SAMEA3541525.psa_megahit.psb_metabat2.00012:k119_4487_5': {'TIGR02121': [[5, 474]]}, 'SAMEA3541525.psa_megahit.psb_metabat2.00012:k119_72410_8': {'TIGR02350': [[3, 574]]}, 'SAMN03287587.psa_megahit.psb_metabat2.00005:k119_46270_3': {'TIGR00659': [[12, 226]]}, 'SAMEA3541525.psa_megahit.psb_metabat2.00012:k119_19044_2': {'TIGR02074': [[68, 609]]}}

In [12]:
for c in final.keys():
    print(c,contig_parts_pos[re.sub('_\d+$','',c)])

SAMEA3541525.psa_megahit.psb_metabat2.00012:k119_54396_4 ['0:950', '+64', '2034:end']
SAMEA2737906.psa_megahit.psb_metabat2.00057:k119_140652_3 ['0:783', '-500', '2513:end']
SAMN03287587.psa_megahit.psb_metabat2.00005:k119_74011_5 ['0:284', '+193', '2765:3239', '+225', '4725:4727', '-365', '8542:end']
SAMN03287620.psa_megahit.psb_metabat2.00029:k119_125320_4 ['0:64', '+365', '3879:3881', '-225', '5349:5803', '-193', '8323:end']
SAMEA3541525.psa_megahit.psb_metabat2.00012:k119_5248_1 ['0:520', '+225', '2025:end']
SAMEA3541525.psa_megahit.psb_metabat2.00012:k119_2244_2 ['0:199', '+171', '2577:end']
SAMN03287620.psa_megahit.psb_metabat2.00029:k119_86524_2 ['0:1', '-262', '1003:1005', '-171', '5499:5501', '+602', '6794:6822', '+291', '9329:end']
SAMEA3541525.psa_megahit.psb_metabat2.00012:k119_45615_5 ['0:2433', '+115', '3843:4002', '+397', '5107:5212', '-197', '7772:end']
SAMEA2737906.psa_megahit.psb_metabat2.00057:k119_71631_12 ['0:962', '-192', '4040:6622', '+115', '8034:8136', '+397', 

In [14]:
n=1
for path in paths:
    for block in path:
        if block=='64A':
            print(n)
            print(path)
            break
    n+=1

13
['217B', '217A', '64A', '64B']


In [16]:
paths[92]

['139A',
 '139B',
 '362A',
 '362B',
 '529A',
 '529B',
 '389A',
 '389B',
 '526B',
 '526A',
 '383A',
 '383B']

In [15]:
print('A',G.nodes['64A'])
print()
print('B',G.nodes['64B'])

A {'cps': {'start': {'SAMN03287620.psa_megahit.psb_metabat2.00029:k119_36935': (None, 425, 1)}, 'end': {'SAMN03287620.psa_megahit.psb_metabat2.00029:k119_36935': (424, None, -1)}, '217A': {'SAMN03287620.psa_megahit.psb_metabat2.00029:k119_36935': (1588, 1588, 1)}}, 'visited': True}

B {'cps': {'start': {'SAMN03287620.psa_megahit.psb_metabat2.00029:k119_36935': (None, 425, 1)}, 'end': {'SAMN03287620.psa_megahit.psb_metabat2.00029:k119_36935': (424, None, -1)}, '217A': {'SAMN03287620.psa_megahit.psb_metabat2.00029:k119_36935': (1588, 1588, 1)}}, 'visited': True}


In [15]:
contig_parts_pos['SAMN03287620.psa_megahit.psb_metabat2.00029:k119_46031']

['0:1',
 '+529',
 '1767:1770',
 '+389',
 '4380:4875',
 '-526',
 '6501:7103',
 '+383',
 '9229:end']

In [ ]:
SAMEA2737906.psa_megahit.psb_metabat2.00057:k119_172198_2 ['0:1', '-111', '2769:end']
SAMN03287620.psa_megahit.psb_metabat2.00029:k119_72255_3 ['0:1485', '-111', '4242:end']
SAMEA3541525.psa_megahit.psb_metabat2.00012:k119_20818_13 ['0:364', '+6', '2805:3269', '-27', '4914:5002', '+234', '7374:7424', '+111', '10079:end']

In [17]:
# sample_fine

cons='/g/scb/bork/groudko/fuso_medium/sibeliaz/mags/sample_fine/alignments/all_cons.fa'
permutations='/g/scb/bork/groudko/fuso_medium/sibeliaz/mags/sample_fine/genomes_permutations.txt'
gff='/g/scb/bork/groudko/fuso_medium/sibeliaz/mags/sample_fine/blocks_coords.gff'
contigs_fasta='/g/scb/bork/groudko/fuso_medium/samples/medium_contigs.fa'
new_contigs_file='/g/scb/bork/groudko/fuso_medium/sibeliaz/mags/sample_fine/alignments/mergedLCB.fasta'

df_gff=df_from_gff(gff)
blocks=dict_with_blocks(cons)
contigs=fasta_to_dict(contigs_fasta)

G=build_graph_cp(permutations,df_gff)
paths=traverse(G)
new=get_contigs(G,paths,contigs)
dict_to_fasta(new,new_contigs_file)

______________________________
1
['861B', '861A', '346A', '346B', '1071A', '1071B', '1213A', '1213B', '212B', '212A', '584A', '584B', '1086A', '1086B', '534B', '534A', '1A', '1B', '262B', '262A', '1414B', '1414A', '343B', '343A', '1238B', '1238A', '1378B', '1378A']
start 861B
SAMN03287620.psa_megahit.psb_metabat2.00029:k119_26869
-
CGTATCAGTT...TATCTTATTC GAATAAGATA...AACTGATACG
current node 346A
is cp 861A 346A {'346A': {'SAMN03287620.psa_megahit.psb_metabat2.00029:k119_26869': (1027, 1027, -1)}, 'end': {'SAMN03287620.psa_megahit.psb_metabat2.00029:k119_26869': (2944, None, 1)}, 'start': {'SAMN03287620.psa_megahit.psb_metabat2.00029:k119_26869': (None, 2943, -1)}}
+
current node 1071A
is cp 346B 1071A {'1071A': {'SAMN03287620.psa_megahit.psb_metabat2.00029:k119_26869': (806, 806, -1)}, '861A': {'SAMN03287620.psa_megahit.psb_metabat2.00029:k119_26869': (1028, 1028, 1)}}
+
current node 1213A
not cp 1071B 1213A {'start': {'SAMN03287620.psa_megahit.psb_metabat2.00029:k119_26869': (None, 0

is cp 1153B 523A {'523A': {'SAMN03287620.psa_megahit.psb_metabat2.00029:k119_63242': (15831, 15397, -1)}, '472A': {'SAMN03287620.psa_megahit.psb_metabat2.00029:k119_63242': (16380, 16522, 1)}}
+
current node 1108A
is cp 523B 1108A {'1108A': {'SAMN03287620.psa_megahit.psb_metabat2.00029:k119_63242': (12413, 12413, -1)}, '1153B': {'SAMN03287620.psa_megahit.psb_metabat2.00029:k119_63242': (15398, 15832, 1)}}
+
current node 84A
is cp 1108B 84A {'84A': {'SAMN03287620.psa_megahit.psb_metabat2.00029:k119_63242': (12106, 12106, -1)}, '523B': {'SAMN03287620.psa_megahit.psb_metabat2.00029:k119_63242': (12414, 12414, 1)}}
+
current node 556A
is cp 84B 556A {'556A': {'SAMN03287620.psa_megahit.psb_metabat2.00029:k119_63242': (11328, 11328, -1)}, '1108B': {'SAMN03287620.psa_megahit.psb_metabat2.00029:k119_63242': (12107, 12107, 1)}}
+
current node 1209B
is cp 556B 1209B {'1209B': {'SAMN03287620.psa_megahit.psb_metabat2.00029:k119_63242': (5666, 5666, -1)}, '84B': {'SAMN03287620.psa_megahit.psb_metab

GACCATTTTC...TAAAAGAATA TATTCTTTTA...GAAAATGGTC
current node 58A
is cp 688A 58A {'952A': {'SAMN03287620.psa_megahit.psb_metabat2.00029:k119_45975': (6691, 6691, -1)}, '58A': {'SAMN03287620.psa_megahit.psb_metabat2.00029:k119_45975': (6934, 6934, 1)}}
+
current node 667A
is cp 58B 667A {'688A': {'SAMN03287620.psa_megahit.psb_metabat2.00029:k119_45975': (6933, 6933, -1)}, '667A': {'SAMN03287620.psa_megahit.psb_metabat2.00029:k119_45975': (7241, 7241, 1)}}
+
current node 103A
is cp 667B 103A {'58B': {'SAMN03287620.psa_megahit.psb_metabat2.00029:k119_45975': (7240, 7240, -1)}, '103A': {'SAMN03287620.psa_megahit.psb_metabat2.00029:k119_45975': (7885, 7885, 1)}}
+
current node 275A
is cp 103B 275A {'667B': {'SAMN03287620.psa_megahit.psb_metabat2.00029:k119_45975': (7884, 7884, -1)}, '275A': {'SAMN03287620.psa_megahit.psb_metabat2.00029:k119_45975': (8091, 8091, 1)}}
+
current node 26A
is cp 275B 26A {'103B': {'SAMN03287620.psa_megahit.psb_metabat2.00029:k119_45975': (8090, 8090, -1)}, '26A':

is cp 437A 65A {'86B': {'SAMN03287620.psa_megahit.psb_metabat2.00029:k119_129120': (644, 644, -1)}, '65A': {'SAMN03287620.psa_megahit.psb_metabat2.00029:k119_129120': (4329, 4329, 1)}}
+
current node 392A
is cp 65B 392A {'437A': {'SAMN03287620.psa_megahit.psb_metabat2.00029:k119_129120': (4328, 4328, -1)}, '392A': {'SAMN03287620.psa_megahit.psb_metabat2.00029:k119_129120': (5031, 5031, 1)}}
+
current node 1222A
is cp 392B 1222A {'65B': {'SAMN03287620.psa_megahit.psb_metabat2.00029:k119_129120': (5030, 5030, -1)}, '1222A': {'SAMN03287620.psa_megahit.psb_metabat2.00029:k119_129120': (5745, 5745, 1)}}
+
current node 385B
not cp 1222B 385B {'392B': {'SAMN03287620.psa_megahit.psb_metabat2.00029:k119_129120': (5744, 5744, -1)}, 'end': {'SAMN03287620.psa_megahit.psb_metabat2.00029:k119_129120': (5965, None, 1)}, 'start': {'SAMN03287620.psa_megahit.psb_metabat2.00029:k119_129120': (None, 5964, -1)}}
-
TTCATCTTCA...TAGGATTAAA TTTAATCCTA...TGAAGATGAA
end 385A
______________________________
52
['

current node 426B
is cp 43A 426B {'start': {'SAMN03287620.psa_megahit.psb_metabat2.00029:k119_47293': (None, 10, 1)}, 'end': {'SAMN03287620.psa_megahit.psb_metabat2.00029:k119_47293': (9, None, -1)}, '426B': {'SAMN03287620.psa_megahit.psb_metabat2.00029:k119_47293': (246, 246, 1)}}
-
CTTCCATATT...AAATCAAACT AGTTTGATTT...AATATGGAAG
current node 866A
is cp 426A 866A {'43A': {'SAMN03287620.psa_megahit.psb_metabat2.00029:k119_47293': (245, 245, -1)}, '866A': {'SAMN03287620.psa_megahit.psb_metabat2.00029:k119_47293': (501, 501, 1)}}
+
current node 421A
is cp 866B 421A {'426A': {'SAMN03287620.psa_megahit.psb_metabat2.00029:k119_47293': (500, 500, -1)}, '421A': {'SAMN03287620.psa_megahit.psb_metabat2.00029:k119_47293': (2144, 2144, 1)}}
+
current node 1248B
not cp 421B 1248B {'866B': {'SAMN03287620.psa_megahit.psb_metabat2.00029:k119_47293': (2143, 2143, -1)}, '1034B': {'SAMN03287620.psa_megahit.psb_metabat2.00029:k119_47293': (2818, 2845, 1)}}
-
ACTGTTACAA...ATTATTTAAA TTTAAATAAT...TTGTAACAG